# Baseline model

## Import modules and load data

In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
data_path = os.path.join(os.getcwd(), '..', 'input')
labels = os.path.join(data_path, 'traininglabels.csv')

In [3]:
df = pd.read_csv(labels)
df.head()

,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


In [27]:
def img_as_array(image_id, size=None, image_set='train_images'):
    image_path = os.path.join(data_path, image_set, image_id)
    img = cv2.imread(str(image_path))
    if size is None:
        return img
    return cv2.resize(img, size)

In [5]:
height = 256
width = 256

In [6]:
images = []
responses = []
scores = []
for idx, row in df.iterrows():
    img = img_as_array(row[0])  #, size=(height, width))
    images.append(img.reshape(1, height, width, 3))
    responses.append(row[1])
    scores.append(row[2])

In [7]:
images = np.concatenate(images, axis=0)
responses = np.array(responses).reshape(images.shape[0], 1)
scores = np.array(scores).reshape(images.shape[0], 1)

In [8]:
print("responses: {}".format(responses.shape))
print("scores: {}".format(scores.shape))
print("images: {}".format(images.shape))

responses: (15244, 1)
scores: (15244, 1)
images: (15244, 256, 256, 3)


## Pickle train set for future usage

*NOTE: Images aren't normalized!*

In [9]:
with open(os.path.join(data_path, 'train_images_256x256.pkl'), 'wb') as fout:
    pickle.dump(images, fout, protocol=4)
with open(os.path.join(data_path, 'train_responses.pkl'), 'wb') as fout:
    pickle.dump(responses, fout, protocol=4)
with open(os.path.join(data_path, 'train_scores.pkl'), 'wb') as fout:
    pickle.dump(scores, fout, protocol=4)

We'll also generate pickles of resized images in case we need them for simpler models:

In [10]:
images64 = []
for idx, row in df.iterrows():
    img = img_as_array(row[0], size=(64, 64))
    images64.append(img.reshape(1, 64, 64, 3))
images64 = np.concatenate(images64, axis=0)
with open(os.path.join(data_path, 'train_images_64x64.pkl'), 'wb') as fout:
    pickle.dump(images64, fout, protocol=4)

In [11]:
images128 = []
for idx, row in df.iterrows():
    img = img_as_array(row[0], size=(128, 128))
    images128.append(img.reshape(1, 128, 128, 3))
images128 = np.concatenate(images128, axis=0)
with open(os.path.join(data_path, 'train_images_128x128.pkl'), 'wb') as fout:
    pickle.dump(images128, fout, protocol=4)

In [12]:
# Delete resized images for now, we won't use them for the baseline
del images64
del images128

## Normalize images

In [13]:
# images = images / 255.
images = images * 2. / 255. - 1.

## Define model architecture

In [22]:
from keras import backend as K
from keras.applications import vgg16
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

In [23]:
class roc_callback(Callback):
    """Define a callback which returns train ROC AUC after each epoch."""

    def __init__(self, training_data, validation_data=None):
        self.x = training_data[0]
        self.y = training_data[1]
        # self.x_val = validation_data[0]
        # self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        # y_pred_val = self.model.predict(self.x_val)
        # roc_val = roc_auc_score(self.y_val, y_pred_val)
        # print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        print('\rroc-auc: {}'.format(round(roc, 5)), end=80 * ' ' + '\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [24]:
def baseline_vgg():
    vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(height, width, 3), pooling='max')
    last = vgg.output
    # x = Flatten()(last)
    x = Dense(128, activation='relu')(last)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=[vgg.input], outputs=[x])

model = baseline_vgg()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [25]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=10, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/10
15244/15244 [==============================] - 122s 8ms/step - loss: 0.0877 - acc: 0.9715
roc-auc: 0.99515                                                                                
Epoch 2/10
15244/15244 [==============================] - 120s 8ms/step - loss: 0.0315 - acc: 0.9901
roc-auc: 0.99863                                                                                
Epoch 3/10
15244/15244 [==============================] - 117s 8ms/step - loss: 0.0267 - acc: 0.9909
roc-auc: 0.99911                                                                                
Epoch 4/10
15244/15244 [==============================] - 114s 7ms/step - loss: 0.0235 - acc: 0.9920
roc-auc: 0.99921                                                                                
Epoch 5/10
15244/15244 [==============================] - 113s 7ms/step - loss: 0.0196 - acc: 0.9937
roc-auc: 0.99948                                                                                
Epoch 6/10

In [26]:
model.save('vgg_baseline_10epoch.h5')

In [55]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-5, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=10, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/10
15244/15244 [==============================] - 121s 8ms/step - loss: 0.0089 - acc: 0.9978
roc-auc: 0.99986                                                                                
Epoch 2/10
15244/15244 [==============================] - 122s 8ms/step - loss: 0.0067 - acc: 0.9982
roc-auc: 0.99987                                                                                
Epoch 3/10
15244/15244 [==============================] - 115s 8ms/step - loss: 0.0064 - acc: 0.9978
roc-auc: 0.99987                                                                                
Epoch 4/10
15244/15244 [==============================] - 114s 7ms/step - loss: 0.0065 - acc: 0.9982
roc-auc: 0.99988                                                                                
Epoch 5/10
15244/15244 [==============================] - 112s 7ms/step - loss: 0.0065 - acc: 0.9981
roc-auc: 0.99989                                                                                
Epoch 6/10

In [56]:
model.save('vgg_baseline_20epoch.h5')

In [63]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-6, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=5, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/5
15244/15244 [==============================] - 116s 8ms/step - loss: 0.0053 - acc: 0.9985
roc-auc: 0.99992                                                                                
Epoch 2/5
15244/15244 [==============================] - 115s 8ms/step - loss: 0.0050 - acc: 0.9986
roc-auc: 0.99992                                                                                
Epoch 3/5
15244/15244 [==============================] - 114s 7ms/step - loss: 0.0050 - acc: 0.9990
roc-auc: 0.99992                                                                                
Epoch 4/5
15244/15244 [==============================] - 114s 7ms/step - loss: 0.0044 - acc: 0.9991
roc-auc: 0.99992                                                                                
Epoch 5/5
15244/15244 [==============================] - 112s 7ms/step - loss: 0.0050 - acc: 0.9986
roc-auc: 0.99992                                                                                


In [64]:
model.save('vgg_baseline_25epoch.h5')

## Load test set and predict

In [72]:
test_dir = 'leaderboard_test_data'
holdout_dir = 'leaderboard_holdout_data'

In [73]:
test_images = []
test_ids = []
for image_id in os.listdir(os.path.join(data_path, test_dir)):
    img = img_as_array(image_id, image_set=test_dir)
    test_images.append(img.reshape(1, height, width, 3))
    test_ids.append(image_id)
for image_id in os.listdir(os.path.join(data_path, holdout_dir)):
    img = img_as_array(image_id, image_set=holdout_dir)
    test_images.append(img.reshape(1, height, width, 3))
    test_ids.append(image_id)
test_images = np.concatenate(test_images, axis=0)

In [74]:
# test_images = test_images / 255.
test_images = test_images * 2. / 255. - 1.

In [75]:
predictions = model.predict(test_images)

In [76]:
predictions = predictions.squeeze().tolist()

In [77]:
with open('submission.csv','w') as fout:
    fout.write("image_id,has_oilpalm\n")
    for image_id, has_oilpalm in zip(test_ids, predictions):
        fout.write("{},{}\n".format(image_id, has_oilpalm))